In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import random
import shutil
from pathlib import Path
import pandas as pd

# Sobel
This code creates the Sobel pre-processed Images

In [ ]:
input_folder = 'pics'          
output_folder = 'sobel'  

os.makedirs(output_folder, exist_ok=True)
for filename in os.listdir(input_folder):
    filepath = os.path.join(input_folder, filename)       
    imag = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if imag is None:
        print(f"Skipping file (could not read): {filename}")
        continue
    sobel_horizontal_kernel = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]])
    img_x = cv2.filter2D(imag, -1, sobel_horizontal_kernel) 
    sobel_vertical_kernel = np.array([[1, 0, -1],[2, 0, -2],[1, 0, -1]])
    img_y = cv2.filter2D(imag, -1, sobel_vertical_kernel) 
    magnitude = np.sqrt(np.power(img_x, 2.0) + np.power(img_y, 2.0))
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = magnitude.astype("uint8")
    
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, magnitude)
    print(f"Processed: {filename} -> {output_path}")

# Creating Labels 
Making sure all pictures have labels. It creates Labels for the Images without faults.

In [ ]:
def create_empty_txts(images_dir, labels_dir):
    
    os.makedirs(labels_dir, exist_ok=True)

    
    png_files = [f for f in os.listdir(images_dir) if f.lower().endswith('.png')]
    created = 0

    for png in png_files:
        base_name = os.path.splitext(png)[0]
        txt_filename = base_name + ".txt"
        txt_path = os.path.join(labels_dir, txt_filename)

        if not os.path.exists(txt_path):
            
            with open(txt_path, 'w') as f:
                pass
            print(f"Created: {txt_filename}")
            created += 1

    print(f"\nDone. {created} empty .txt file(s) created in '{labels_dir}'.")

images_directory = "./pics"   
labels_directory = "./labels"   
create_empty_txts(images_directory, labels_directory)



# Splitting

Splitting the original images and the labels

In [ ]:
np.random.seed(777)


images_dir = Path("pics")
labels_dir = Path("labels")
 

output_dir = Path("dataset")


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15


for split in ['train', 'val', 'test']:
    (output_dir / 'images' / split).mkdir(parents=True, exist_ok=True)
    (output_dir / 'labels' / split).mkdir(parents=True, exist_ok=True)


image_files = list(images_dir.glob("*.png"))
random.shuffle(image_files)


n = len(image_files)
n_train = int(n * train_ratio)
n_val = int(n * val_ratio)
n_test = n - n_train - n_val

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train + n_val]
test_files = image_files[n_train + n_val:]


def split(files, split):
    for img_path in files:
        img_name = img_path.stem
        label_path = labels_dir / f"{img_name}.txt"

        
        shutil.copy(img_path, output_dir / 'images' / split / img_path.name)

        
        if label_path.exists():
            shutil.copy(label_path, output_dir / 'labels' / split / label_path.name)
        else:
            print(f" Label not found for {img_path.name}, skipping.")


split(train_files, 'train')
split(val_files, 'val')
split(test_files, 'test')

copies the same split to the pre-processed Images

In [ ]:
original_split_dir = Path("dataset")  
preprocessed_images_dir = Path("sobel")  
output_dir = Path("dataset_split_preprocessed")  


for splits in ['train', 'val', 'test']:
    (output_dir / 'images' / splits).mkdir(parents=True, exist_ok=True)


def copy_preprocessed(splits):
    original_split_path = original_split_dir / 'images' / splits
    for img_path in original_split_path.iterdir():
        preproc_path = preprocessed_images_dir / img_path.name
        if preproc_path.exists():
            shutil.copy(preproc_path, output_dir / 'images' / splits / img_path.name)
        else:
            print(f"Missing preprocessed file for: {img_path.name}")


for splits in ['train', 'val', 'test']:
    copy_preprocessed(splits)

# Create the split label folders
original_labels_dir = Path("dataset/labels")
output_labels_dir = Path("dataset_split_preprocessed/labels")

# Create the split label folders
for split in ['train', 'val', 'test']:
    (output_labels_dir / split).mkdir(parents=True, exist_ok=True)
    for label_path in (original_labels_dir / split).iterdir():
        shutil.copy(label_path, output_labels_dir / split / label_path.name)


# Training
This trains the Model with the original Images.

Please delete *device=0* if cuda is not active.

In [ ]:
np.random.seed(777)
def training():
    
    model = YOLO('yolov8n.pt')  

    # Train it
    model.train(
        data='data.yaml',
        epochs=100,
        imgsz=640,
        batch=8,
        lr0=0.001,       
        lrf=0.01,        
        momentum=0.937,  
        weight_decay=0.0005,
        warmup_epochs=3,
        warmup_momentum=0.8,
        box=0.05,        
        cls=0.1,         
        dfl=1.5,         
        device=0 #delete line if cuda is not installed
        
    )
training()


This trains the Model with the pre-processed Images

Please delete *device=0* if cuda is not active.

In [ ]:
np.random.seed(777)
def train_sobel():
    
    model = YOLO('yolov8n.pt')  

    
    model.train(
        data='sobel.yaml',
        epochs=100,
        imgsz=640,
        batch=8,
        lr0=0.001,       # initial learning rate
        lrf=0.01,        # final learning rate fraction
        momentum=0.937,  # optimizer momentum
        weight_decay=0.0005,
        warmup_epochs=3,
        warmup_momentum=0.8,
        box=0.05,        # how much are the boxes valued
        cls=0.1,         # only one class= small
        dfl=1.5,         # if object or not
        device=0          #delete line if cuda is not installed
        
    )
train_sobel()

# Testing
This test the model trained by the original Images

Please delete *device=0* if cuda is not active.

In [ ]:
def testing():
    model = YOLO("runs/detect/Normaltrain/weights/best.pt")
    metrics = model.val(
        data="data.yaml",
        split="test",
        device=0,  #delete line if cuda is not installed
        imgsz=640
    )
    print(metrics)
    df = pd.DataFrame([metrics.results_dict])
    df.to_csv("test_metrics.csv", index=False)
testing()

This test the model trained by the pre-processed Images

Please delete *device=0* if cuda is not active.

In [ ]:
def testing():
    model = YOLO("runs/detect/Sobeltrain/weights/best.pt")
    metrics = model.val(
        data="sobel.yaml",
        split="test",
        device=0, #delete line if cuda is not installed
        imgsz=640
    )
    print(metrics)
    df = pd.DataFrame([metrics.results_dict])
    df.to_csv("test_metrics.csv", index=False)
testing()